In [2]:
from smart.datasets.preprocess import TokenProcessor  
import os  
import pickle  
from tqdm import tqdm  # 导入tqdm库  
import shutil

/share/home/tj24005/.conda/envs/smart/lib/python3.8/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: libcusparse.so.11: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
# raw_dir = "/ssdfs/datahome/tj24005/mini_smart_waymo_processed"
# process_dir = "/ssdfs/datahome/tj24005/mini_smart_waymo_processed/process"
# move_target_dir = "/share/home/tj24005/data/mini_smart_waymo_processed"

raw_dir = "/ssdfs/datahome/tj24005/smart_waymo_processed"
process_dir = "/ssdfs/datahome/tj24005/smart_waymo_processed/process"
move_target_dir = "/share/home/tj24005/data/smart_waymo_processed"

In [4]:
files = []

# 加载所有待处理的数据
for split in ['training', 'validation', 'testing']:  
    os.makedirs(os.path.join(process_dir, split), exist_ok=True)
    os.makedirs(os.path.join(move_target_dir, split), exist_ok=True)
    for f in os.listdir(os.path.join(raw_dir, split)):
        files.append((split, f))

In [6]:
def split_list_into_parts(lst, num_parts):
    # 确保分成 num_parts 等份
    avg_len = len(lst) // num_parts
    remainder = len(lst) % num_parts
    
    result_dict = {}
    start = 0
    
    for i in range(num_parts):
        # 如果有剩余，前 remainder 份分配一个额外的元素
        end = start + avg_len + (1 if i < remainder else 0)
        result_dict[i] = lst[start:end]
        start = end

    return result_dict

tasks = split_list_into_parts(files, 48)

for k, v in tasks.items():
    print(k, len(v))

with open("scripts/data_process.pkl", 'wb') as handle:
    pickle.dump(tasks, handle)

0 12001
1 12001
2 12001
3 12000
4 12000
5 12000
6 12000
7 12000
8 12000
9 12000
10 12000
11 12000
12 12000
13 12000
14 12000
15 12000
16 12000
17 12000
18 12000
19 12000
20 12000
21 12000
22 12000
23 12000
24 12000
25 12000
26 12000
27 12000
28 12000
29 12000
30 12000
31 12000
32 12000
33 12000
34 12000
35 12000
36 12000
37 12000
38 12000
39 12000
40 12000
41 12000
42 12000
43 12000
44 12000
45 12000
46 12000
47 12000


In [7]:
with open("scripts/data_process.pkl", 'rb') as handle:
    tasks = pickle.load(handle)
task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', 0))
# num_tasks = int(os.getenv('SLURM_ARRAY_TASK_COUNT', 1))
files_to_process = tasks[task_id]

In [8]:
(

[('training', '15e6caf92c5cb188.pkl'),
 ('training', '17dfbe60bbb837d7.pkl'),
 ('training', '19ea1cacac7867f8.pkl'),
 ('training', '21226a8c0eff561.pkl'),
 ('training', '216b9ef5e5d5c3d7.pkl'),
 ('training', '22b7afbfa5f4997d.pkl'),
 ('training', '2304b44970a374.pkl'),
 ('training', '33d8eaa63ac1926e.pkl'),
 ('training', '3cde711b9173da56.pkl'),
 ('training', '48d4f66482bf0dd3.pkl'),
 ('training', '494844639bb933c8.pkl'),
 ('training', '5279e6a7b20552dd.pkl'),
 ('training', '5eb54a1ff74d8823.pkl'),
 ('training', '5f8dfb6b61d0adcc.pkl'),
 ('training', '703554c0e1e984fe.pkl'),
 ('training', '7e2618e090194942.pkl'),
 ('training', '7f6c8bee44cf218e.pkl'),
 ('training', '8494e61cf30b038b.pkl'),
 ('training', '885c1b2046d7442d.pkl'),
 ('training', '8d97684ec13e465b.pkl'),
 ('training', 'a780c0f9e26ff951.pkl'),
 ('training', 'b09802fbd9fedf1b.pkl'),
 ('training', 'b76f3703395ab491.pkl'),
 ('training', 'cded046329527857.pkl'),
 ('training', 'd5b763cd4eff35f7.pkl'),
 ('training', 'de4e1315e4c3a

In [31]:
notify = [len(files_to_process) * (i+1) // 100 for i in range(0, 100, 1)]

In [37]:
token_processor = TokenProcessor(2048) 

# 使用tqdm创建进度条  
# with tqdm(total=len(files_to_process), desc=f"Task {task_id} Processing files") as pbar:  
for i, (split, f) in enumerate(files_to_process):
    if i+1 in notify:
        print(f"Task {task_id} Processing {i+1}/{len(files_to_process)}({(i+1)/len(files_to_process)*100:.2f}%)")
    raw_file_path = os.path.join(raw_dir, split, f)
    process_file_path = os.path.join(process_dir, split, f)
    move_file_path = os.path.join(move_target_dir, split, f)

    if os.path.exists(raw_file_path):
        with open (raw_file_path, 'rb') as handle:
            raw_data = pickle.load(handle)
        process_data = token_processor.preprocess(raw_data)
        with open(process_file_path, 'wb') as handle:  
            pickle.dump(process_data, handle)
        try:  
            shutil.move(raw_file_path, move_file_path)
        except Exception as e:  
            print(f"移动文件时出错: {e}")
        
        # pbar.update(1)



Task 0 Processing 2/240(0.83%)
Task 0 Processing 4/240(1.67%)
Task 0 Processing 7/240(2.92%)
Task 0 Processing 9/240(3.75%)
Task 0 Processing 12/240(5.00%)
Task 0 Processing 14/240(5.83%)
Task 0 Processing 16/240(6.67%)
Task 0 Processing 19/240(7.92%)
Task 0 Processing 21/240(8.75%)
Task 0 Processing 24/240(10.00%)
Task 0 Processing 26/240(10.83%)
Task 0 Processing 28/240(11.67%)
Task 0 Processing 31/240(12.92%)
Task 0 Processing 33/240(13.75%)
Task 0 Processing 36/240(15.00%)
Task 0 Processing 38/240(15.83%)
Task 0 Processing 40/240(16.67%)
Task 0 Processing 43/240(17.92%)
Task 0 Processing 45/240(18.75%)
Task 0 Processing 48/240(20.00%)
Task 0 Processing 50/240(20.83%)
Task 0 Processing 52/240(21.67%)
Task 0 Processing 55/240(22.92%)
Task 0 Processing 57/240(23.75%)
Task 0 Processing 60/240(25.00%)
Task 0 Processing 62/240(25.83%)
Task 0 Processing 64/240(26.67%)
Task 0 Processing 67/240(27.92%)
Task 0 Processing 69/240(28.75%)
Task 0 Processing 72/240(30.00%)
Task 0 Processing 74/24